<a href="https://colab.research.google.com/github/horger89/QuestionAnsweringBert/blob/main/QuestionAnsweringBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00


In [2]:
from huggingface_hub import notebook_login

notebook_login()

Load SQuAD dataset

In [3]:
from datasets import load_dataset

squad = load_dataset("squad", split="train[:35000]")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Split the dataset’s train split into a train and test set

In [4]:
squad = squad.train_test_split(test_size=0.2)

Take a look at an example

In [5]:
squad["train"][0]

{'id': '56d9e7e4dc89441400fdb903',
 'title': 'Dog',
 'context': 'A Colorado study found bites in children were less severe than bites in adults. The incidence of dog bites in the US is 12.9 per 10,000 inhabitants, but for boys aged 5 to 9, the incidence rate is 60.7 per 10,000. Moreover, children have a much higher chance to be bitten in the face or neck. Sharp claws with powerful muscles behind them can lacerate flesh in a scratch that can lead to serious infections.',
 'question': 'About 12.9 out of 10,000 are bit by dogs, but what is the number in 10,000 for young boys from 5 to 9?',
 'answers': {'text': ['60.7'], 'answer_start': [197]}}

Load a DistilBERT tokenizer to process the question and context fields

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

The function to truncate and map the start and end tokens of the answer to the context

In [7]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

Apply the preprocessing function over the entire dataset

In [8]:
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

Map:   0%|          | 0/28000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

Create a batch of examples using DefaultDataCollator

In [9]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator(return_tensors="tf")

 Setting up an optimizer function, learning rate schedule, and some training hyperparameters

In [11]:
from transformers import create_optimizer

batch_size = 16
num_epochs = 3
total_train_steps = (len(tokenized_squad["train"]) // batch_size) * num_epochs
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=total_train_steps,
)

Load DistilBERT with TFAutoModelForQuestionAnswering

In [12]:
from transformers import TFAutoModelForQuestionAnswering

model = TFAutoModelForQuestionAnswering.from_pretrained("distilbert/distilbert-base-uncased")

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForQuestionAnswering: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForQuestionAnswering were not initialized from the PyTorch model and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it

Convert datasets to the tf.data.Dataset format

In [13]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_squad["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_squad["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

Before start training here is a way to push your model to the Hub

In [14]:
from transformers.keras_callbacks import PushToHubCallback

callback = PushToHubCallback(
    output_dir="SlightlyFineTunedBertOnSquad",
    tokenizer=tokenizer,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/horger89/SlightlyFineTunedBertOnSquad into local empty directory.


Download file tf_model.h5:   0%|          | 8.00k/253M [00:00<?, ?B/s]

Clean file tf_model.h5:   0%|          | 1.00k/253M [00:00<?, ?B/s]

Configure the model for training with compile

In [15]:
import tensorflow as tf

model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

Training the model

In [ ]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3, callbacks=[callback])

Epoch 1/3


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1750/1750 [==============================] - 1366s 752ms/step - loss: 8.3613 - end_logits_loss: 4.1160 - start_logits_loss: 4.2453 - end_logits_accuracy: 0.1252 - start_logits_accuracy: 0.1231 - val_loss: 7.2387 - val_end_logits_loss: 3.6778 - val_start_logits_loss: 3.5609 - val_end_logits_accuracy: 0.2866 - val_start_logits_accuracy: 0.2680
Epoch 2/3
1750/1750 [==============================] - 1289s 737ms/step - loss: 7.4922 - end_logits_loss: 3.5571 - start_logits_loss: 3.9351 - end_logits_accuracy: 0.2865 - start_logits_accuracy: 0.2273 - val_loss: 6.9356 - val_end_logits_loss: 3.5662 - val_start_logits_loss: 3.3694 - val_end_logits_accuracy: 0.3823 - val_start_logits_accuracy: 0.3239
Epoch 3/3
1750/1750 [==============================] - 1287s 735ms/step - loss: 6.5399 - end_logits_loss: 3.1546 - start_logits_loss: 3.3853 - end_logits_accuracy: 

Inference

In [1]:
question = "Where is the Nile River's headwaters located?"
context = "The Nile is the longest river in Africa and the longest river in the world by measured length. It is 6,650 kilometers (4,140 mi) in length. The Nile's headwaters are in the mountains of Burundi and Rwanda. It flows north to Egypt, where it empties into the Mediterranean Sea. The Nile Basin is home to eleven countries: Burundi, Rwanda, Democratic Republic of the Congo, United Republic of Tanzania, Kenya, Uganda, South Sudan, Ethiopia, Eritrea, Sudan, and Egypt."

Tokenize the text and return TensorFlow tensors

In [ ]:
inputs = tokenizer(question, context, return_tensors="tf")

Pass your inputs to the model and return the logits

In [ ]:
outputs = model(**inputs)

Get the highest probability from the model output for the start and end positions

In [ ]:
answer_start_index = int(tf.math.argmax(outputs.start_logits, axis=-1)[0])
answer_end_index = int(tf.math.argmax(outputs.end_logits, axis=-1)[0])

Decode the predicted tokens to get the answer

In [ ]:
predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens)